In [0]:
CREATE OR REPLACE TEMP VIEW tx_1mln AS
SELECT
  id                                AS txn_id,
  -- Descending timestamps: each next row is 1 minute *earlier*
  timestampadd(MINUTE, -(1000000 - id), current_timestamp()) AS txn_ts,
  CAST(rand()*100000 AS INT)         AS account_id,
  ROUND(rand()*1000, 2)              AS amount,
  CASE WHEN rand() < 0.95 THEN 'purchase' ELSE 'refund' END AS txn_type,
  SUBSTR(uuid(), 1, 12)              AS note
FROM range(0, 1000000, 1, 2000);


In [0]:
DROP TABLE IF EXISTS transactions_managed;

CREATE TABLE transactions_managed
(
  txn_id     BIGINT,
  txn_ts     TIMESTAMP,
  account_id INT,
  amount     DECIMAL(10,2),
  txn_type   STRING,
  note       STRING,
  txn_date   DATE GENERATED ALWAYS AS (CAST(txn_ts AS DATE))
)
PARTITIONED BY (txn_date)
TBLPROPERTIES (
  delta.autoOptimize.autoCompact = false,
  delta.autoOptimize.optimizeWrite = false
);

-- Insert 1M rows; settings from step 1 enforce tiny files.
INSERT INTO transactions_managed (txn_id, txn_ts, account_id, amount, txn_type, note)
SELECT * FROM tx_1mln;


In [0]:
-- We'll land the external table under a subfolder of your container:
-- Make sure the path is under the external location you created in step 0.
-- Example folder: abfss://unity@westus2uc.dfs.core.windows.net/uc_vs_ext/transactions_ext

DROP TABLE IF EXISTS transactions_external;

CREATE TABLE transactions_external
(
  txn_id     BIGINT,
  txn_ts     TIMESTAMP,
  account_id INT,
  amount     DECIMAL(10,2),
  txn_type   STRING,
  note       STRING,
  txn_date   DATE GENERATED ALWAYS AS (CAST(txn_ts AS DATE))
)
PARTITIONED BY (txn_date)
LOCATION 'abfss://unity@westus2uc.dfs.core.windows.net/transactions_ext'
TBLPROPERTIES (
  delta.autoOptimize.autoCompact = false,
  delta.autoOptimize.optimizeWrite = false
);

-- Insert identical data to the external table
INSERT INTO transactions_external (txn_id, txn_ts, account_id, amount, txn_type, note)
SELECT * FROM tx_1mln;


In [0]:
SET use_cached_result = false;  

In [0]:
SELECT COUNT(*) FROM transactions_managed;

In [0]:
SELECT COUNT(*) FROM transactions_external;

Log analityc query after setting sending diagnostics to Log Workspace:
```
StorageBlobLogs
| where TimeGenerated > ago(30m)
| project TimeGenerated,
          OperationName,
          ResponseHeaderSize,
          ResponseBodySize
| order by TimeGenerated desc
```